In [89]:
import sklearn as skl
import pandas as pd
import numpy as np

%matplotlib inline

In [90]:
df_156 = pd.read_hdf('../../data/processed/solicitacoes156.h5', key='data', format="table")
df_plu = pd.read_hdf('../../data/processed/pluviometrico.h5', key='data', format="table")

In [91]:
filtered_df = df_156[df_156['SERVICO_DESCRICAO'] == 'QUEDA DE ARVORE']
group_df = filtered_df.groupby('DATA_DEMANDA')[['SERVICO_CODIGO']]
count_df = group_df.count().reset_index()
count_df.columns = ['DATA', 'QUEDAS']
df = df_plu.merge(count_df, on='DATA')

In [92]:
month_df = pd.get_dummies(df['DATA'].dt.month)
month_df.columns = ['m' + str(i) for i in range(1, 13)]

day_df = pd.get_dummies(df['DATA'].dt.day)
day_df.columns = ['d' + str(i) for i in range(1, 32)]

df = pd.concat([day_df, month_df, df[['INDICE', 'QUEDAS']]], axis=1)

In [93]:
from sklearn.neighbors import KNeighborsRegressor
skl.utils.shuffle(df)

split = int(df.shape[0] * 0.7)
train, test = df.iloc[:split,:], df.iloc[split:,:]

clf = KNeighborsRegressor(
    weights='distance',
    n_neighbors=5)

In [100]:
clf.fit(train.iloc[:,:-1], train.iloc[:,-1])

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=5, p=2,
          weights='distance')

In [101]:
clf.score(test.iloc[:,:-1], test.iloc[:,-1])

-0.013196116581627448

In [102]:
skl.metrics.mean_absolute_error(clf.predict(test.iloc[:,:-1]), test.iloc[:,-1])

1.89753285953115

In [97]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [98]:
formula = 'QUEDAS ~ d1 + d2 + d3 + d4 + d5 + d6 + d7 + d8 + d9 + d10 + d11 + d12 + d13 + d14 + d15 + d16 + d17 + d18 + d19 + d20 + d21 + d22 + d23 + d24 + d25 + d26 + d27 + d28 + d29 + d30 + d31 + m1 + m2 + m3 + m4 + m5 + m6 + m7 + m8 + m9 + m10 + m11 + m12 + INDICE'

In [99]:
model = smf.ols(formula,data=df)
model.fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 QUEDAS   R-squared:                       0.068
Model:                            OLS   Adj. R-squared:                  0.035
Method:                 Least Squares   F-statistic:                     2.051
Date:                Thu, 29 Nov 2018   Prob (F-statistic):           0.000109
Time:                        03:52:52   Log-Likelihood:                -3809.5
No. Observations:                1229   AIC:                             7705.
Df Residuals:                    1186   BIC:                             7925.
Df Model:                          42                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.6596      0.161     10.340      0.000       1.345       1.974
d1             0.1556      0.999      0.156      0.876      -1.805       2.116
d2            -0.1384      0.797     -0.174      0.862      -1.702       1.425
d3            -0.2523      0.874     -0.289      0.773      -1.966       1.462
d4             0.6704      0.863      0.777      0.437      -1.023       2.364
d5             0.2124      0.842      0.252      0.801      -1.440       1.865
d6            -0.3905      0.814     -0.480      0.632      -1.988       1.207
d7             0.0504      0.875      0.058      0.954      -1.667       1.767
d8            -0.0476      0.885     -0.054      0.957      -1.784       1.689
d9             0.2627      0.780      0.337      0.736      -1.268       1.793
d10           -0.3946      0.841     -0.469      0.639      -2.045       1.256
d11           -0.0545      0.806     -0.068      0.946      -1.637       1.528
d12            0.0738      0.898      0.082      0.934      -1.687       1.835
d13            0.7934      0.843      0.941      0.347      -0.860       2.447
d14           -0.2781      0.951     -0.292      0.770      -2.144       1.588
d15           -0.2599      0.875     -0.297      0.766      -1.976       1.456
d16            0.1446      0.910      0.159      0.874      -1.641       1.930
d17            0.3780      0.833      0.454      0.650      -1.256       2.012
d18           -0.0929      0.832     -0.112      0.911      -1.725       1.539
d19           -0.0149      0.966     -0.015      0.988      -1.911       1.881
d20           -0.4314      0.780     -0.553      0.580      -1.962       1.099
d21           -0.2250      0.876     -0.257      0.797      -1.943       1.493
d22           -0.3667      0.833     -0.440      0.660      -2.001       1.268
d23           -0.5110      0.842     -0.607      0.544      -2.164       1.142
d24           -0.5358      0.833     -0.643      0.520      -2.170       1.098
d25           -0.7010      0.822     -0.853      0.394      -2.314       0.912
d26           -0.2451      0.833     -0.294      0.769      -1.879       1.388
d27           -0.4956      0.816     -0.608      0.544      -2.096       1.105
d28           -0.4997      0.898     -0.557      0.578      -2.261       1.262
d29            3.7221      0.845      4.403      0.000       2.063       5.381
d30            1.4890      0.827      1.800      0.072      -0.134       3.112
d31           -0.3580      1.058     -0.338      0.735      -2.433       1.717
m1             2.2845      0.516      4.427      0.000       1.272       3.297
m2             0.2506      0.509      0.492      0.623      -0.749       1.250
m3             0.2257      0.468      0.482      0.630      -0.693       1.144
m4             0.3915      0.487      0.804      0.421      -0.564       1.347
m5